In [1]:
from __future__ import division
import pandas as pd
import os
import numpy as np
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

### Creates design files for the instrumental learning task

An experimental session containts two task, each of which is performed in three runs: 1. a vanilla instrumental learning task, and 2. a SAT-version of the task. Task order is counterbalanced, stimuli are updated between tasks. Stimuli are assigned a value randomly (i.e. also counterbalanced).


##### Design files are DataFrames with the following info per trial:
1. `stim_set`: Which stimulus set is presented? [0, 1, 2, 3, 4, 5, 6]
2. `correct_stim_lr`: What is the location (on the screen) of the winning stim? [0 = left, 1=right]
3. `p_win_left`: Probability of winning if left is chosen
4. `p_win_right'`: Probability of winning if right is chosen
5. `p_win_correct`: Probability of winning if correct answer is chosen
6. `p_win_incorrect`: Probability of winning if incorrect answer is chosen
7. `phase_x`: Duration for phase `x` (in s; see below)
8. `stim_left`, `stim_right`:  Stimuli that are presented left, right
9. `stim_high`, `stim_low`: Stimuli that correspond to the high, low probability of winning

##### Trial phases:
1. Fixation cross (jittered timing) `[0.5, 0.75, 1.0, 1.25, 1.5s]`
2. Cue (0.75s)
3. Fixation cross (jittered timing) `[0.5, 0.75, 1.0, 1.25, 1.5s]`
4. Stimulus (2s)
5. Stimulus choice (jittered timing) `[0.5, 0.75, 1.0, 1.25, 1.5s]`
6. Feedback (0.5s)
7. ITI (jittered timing) `[0.25, 0.5, ..., 3.25s]`

The timings mentioned above are for the MR version of the SAT task. Without MR, the jittered fixation crosses are skipped, the stimulus duration is not fixed to 2 seconds but ends when the participant gave a response, and highlight of choice is a fixed duration. ITI is shortened but still jittered a bit.

##### Potential future options:
1. Pseudorandomize trial order;
2. Optimize design (have code for this)

In [2]:
def get_settings(tr=2, verbose=True):
    p_win = [[.8, .2], [.7, .3], [.65, .35]]
    n_runs = 3
    n_sessions = 2
    if tr == 2:
        n_trials = 128
        jitters = [0.5, 0.75, 1, 1.25, 1.5]
        volumes_per_trial = 4
    
    n_trials_per_stimset = n_trials/len(p_win)
    trial_duration = volumes_per_trial*tr
    total_duration = trial_duration*n_trials
    total_duration_min = total_duration/60
    total_volumes = 1 + n_trials*volumes_per_trial
    
    if verbose:
        print('Settings:\n\n\
        Sessions: {n_sessions}\n\
        Trials per run: {n_trials}\n\
        Assuming a TR of {tr} seconds\n\
        Jitter options: {jitters} seconds\n\
        Total duration: {tr}*{trial_duration}*{n_trials} = {total_duration} seconds = {total_duration_min} min\n\
        Total number of volumes necessary: 1+{n_trials}*{volumes_per_trial} = {total_volumes} + warm-up pulses'.format(**locals()))
        
    return({'jitter': jitters,
            'n_trials': n_trials,
            'ps': p_win})

In [18]:
def generate_block_design(n_trials, jitters, 
                          ps,
                          stim_sets=[0,1,2],
                          mr_design=True, 
                          include_cue=True, 
                          phase_durations=['jittered', 0.75, 'jittered', 2, 'jittered', 0.75, 'iti'],
                          trial_duration=8,
                          practice=False):
    """ Generates design for a single block.
    
    jitters: list of possible jitter durations, in seconds (e.g., [0.5, 1, 1.5])
    mr_design: bool. If False, all phases that are jittered are set to -1 and skipped in the experiment; iti is set lower
    include_cue: bool. If False, the cue phase duration will be set to -1.
    """
    correct_stim_lr = [0, 1]
    if include_cue:
        cues = ['SPD', 'ACC']
    else:
        cues = ['']
    combs = list(itertools.product(stim_sets, correct_stim_lr, cues))
    
    # make basic df
    design = pd.DataFrame(combs * int(np.ceil((n_trials/len(combs)))), 
                          columns=['stimulus_set', 'correct_stim_lr', 'cue'])
    # randomize
    design = design.sample(frac=1).reset_index(drop=True)#.reset_index('trial_ID')
    n_trials_real = design.shape[0]
    design = design.iloc[:n_trials]
    
    if not n_trials == n_trials_real:
        print('WARNING: not totally balanced (%d not a multitude of %d)' %(n_trials, len(combs)))
    
    # Add probabilities (left/right and correct/incorrect - this is redundant, I know)
    design['p_win_left'] = None
    design['p_win_right'] = None
    design['p_win_correct'] = None
    design['p_win_incorrect'] = None
    for stim_set, p_win in zip(stim_sets, ps):
        p_win_high, p_win_low = p_win[0], p_win[1]
        design.loc[(design.stimulus_set==stim_set) & (design.correct_stim_lr == 0), 'p_win_left'] = p_win_high
        design.loc[(design.stimulus_set==stim_set) & (design.correct_stim_lr == 1), 'p_win_right'] = p_win_high
        design.loc[(design.stimulus_set==stim_set) & (design.correct_stim_lr == 1), 'p_win_left'] = p_win_low
        design.loc[(design.stimulus_set==stim_set) & (design.correct_stim_lr == 0), 'p_win_right'] = p_win_low
        design.loc[(design.stimulus_set==stim_set), 'p_win_correct'] = p_win_high
        design.loc[(design.stimulus_set==stim_set), 'p_win_incorrect'] = p_win_low
    
    # Add phase durations
    for phase, duration in enumerate(phase_durations):
        col_key = 'phase_' + str(phase+1)
        if duration == 'jittered':
            design[col_key] = np.random.choice(jitters, size=n_trials, replace=True)
        elif duration == 'iti':
            iti_phase_col_key = 'phase_' + str(phase+1)
        else:
            design[col_key] = duration
    
    design[iti_phase_col_key] = trial_duration - design[[col for col in design.columns if 'phase' in col]].apply(sum, axis=1)

    
    if not mr_design:
        # remove jitters
        for phase, duration in enumerate(phase_durations):
            col_key = 'phase_' + str(phase+1)
            if duration == 'jittered':
                design[col_key] = -.0001
            # set iti to randomly sampled from [0.5, 1]
            if duration == 'iti':
                design[col_key] = np.random.uniform(0.5, 1, design.shape[0])
        # set choice highlight phase to fixed 0.5
        design['phase_5'] = 0.5

    if not include_cue:
        # always skip cue
        design['phase_2'] = -.0001
        
    if practice and include_cue:
        design.cue.iloc[0] = 'ACC'

    return(design)

#generate_block_design(18, [0, 1, 2])

In [19]:
def get_task_type(run, subject_id, is_practice=False, is_debug=False):    
    def is_number(s):
        try:
            float(s)
            return True
        except ValueError:
            return False
    
    if is_debug:
        if run > 3:
            include_cue = False
        else:
            include_cue = True
    elif is_practice:
        # start practice without cues
        if run <= 3:
            include_cue = False
        else:
            include_cue = True

    elif is_number(subject_id):
        if (int(subject_id) % 2) == 0:
            if run > 3:
                include_cue = False
            else:
                include_cue = True
        else:
            if run > 3:
                include_cue = True
            else:
                include_cue = False
                
    # Return stimuli
    if run <= 3:
        stim_sets = [0,1,2]
    elif run > 3:
        stim_sets = [3,4,5]
            
    return include_cue, stim_sets

In [20]:
def counterbalance_stimuli(subject_id, all_sets=[['A', 'B'],
                                                 ['J', 'K'], 
                                                 ['M', 'N'],
                                                 ['O', 'P'],
                                                 ['W', 'X'],
                                                 ['c', 'e']]):
        import itertools
        from copy import deepcopy

        n_shifts = [0, 1, 2]  # assuming 6 stimuli, but you could do more, or less...
        rev_inner = [False, True]
        switch_sets = [False, True]

        cb_df = pd.DataFrame(list(itertools.product(switch_sets, n_shifts, rev_inner)),
                             columns=['switch_sets', 'n_shifts', 'rev_inner'])
        cb_df['pp'] = np.arange(1, cb_df.shape[0] + 1)
        for set_n in range(1, 13):
            cb_df['stim_%d' % set_n] = None

        for pp in cb_df['pp']:
            idx = cb_df.pp == pp
            switch_sets = cb_df.loc[idx, 'switch_sets'].iloc[0]
            reverse_inner = cb_df.loc[idx, 'rev_inner'].iloc[0]
            n_shifts = cb_df.loc[idx, 'n_shifts'].iloc[0]

            if switch_sets:
                sets = deepcopy([all_sets[3:], all_sets[:3]])
            else:
                sets = deepcopy([all_sets[:3], all_sets[3:]])

            sets_allocated = 0
            for set_n, set_ in enumerate(sets):
                for i in range(n_shifts):
                    set_.insert(len(set_), set_.pop(0))  # move first item to last place

                if reverse_inner:
                    set_ = [x[::-1] for x in set_]  # reverse inner order

                # print('pp %d, %d, %s' % (pp, set_n, set_))
                #### NB: you could just use set_ as a final result; the placing in the dataframe and then reverting
                # back to a nested list is definitely not necessary but may help clarify what's going on here...
                for to_allocate in [0, 1, 2]:
                    for to_allocate_i in [0, 1]:
                        cb_df.loc[idx, 'stim_%d' % (sets_allocated + 1)] = set_[to_allocate][to_allocate_i]
                        sets_allocated += 1

        pp_zero_based = int(subject_id) - 1
        row_iloc = int(pp_zero_based - np.floor(pp_zero_based / 12) * 12)
        colnames = cb_df.columns
        stim_list = cb_df.iloc[row_iloc][[x for x in colnames if 'stim' in x]].values.tolist()
        stim_nested_list = [[stim_list[0 + y * 2], stim_list[1 + y * 2]] for y in range(6)]
        print('Stimuli/set order for this pp: %s' % stim_nested_list)
        return stim_nested_list


In [21]:
def get_stimuli(subject_id, design):    
    if subject_id == 'PRACTICE':
        # The practice session always gets different stimuli
        stim_sets = [['j', 'm'], ['s', 'y'], ['u', 'n'], ['j', 'm'], ['s', 'y'], ['u', 'n']]
    else:
        stim_sets = [['A', 'B'], ['J', 'K'], ['M', 'N'], ['O', 'P'],  ['W', 'X'], ['c', 'e']]
    if subject_id == 'DEBUG' or subject_id == 'PRACTICE':
        # just use counterbalancing order 1 for practice & debug
        subject_id = 1

    all_stim = counterbalance_stimuli(subject_id, all_sets=stim_sets)
    design['stim_left'] = None
    design['stim_right'] = None
    design['stim_high'] = None
    design['stim_low'] = None

    for i in range(len(all_stim)):
        idx_this_stim = design.stimulus_set == i
        design.loc[idx_this_stim, 'stim_high'] = all_stim[i][0]
        design.loc[idx_this_stim, 'stim_low'] = all_stim[i][1]

    for i in range(len(all_stim)):
        idx_this_stim = design.stimulus_set == i
        idx_this_stim_nonrev = (idx_this_stim) & (design.correct_stim_lr == 0)
        idx_this_stim_rev = (idx_this_stim) & (design.correct_stim_lr == 1)

        design.loc[idx_this_stim_nonrev, 'stim_left'] = design.loc[idx_this_stim_nonrev, 'stim_high']
        design.loc[idx_this_stim_nonrev, 'stim_right'] = design.loc[idx_this_stim_nonrev, 'stim_low']
        design.loc[idx_this_stim_rev, 'stim_left'] = design.loc[idx_this_stim_rev, 'stim_low']
        design.loc[idx_this_stim_rev, 'stim_right'] = design.loc[idx_this_stim_rev, 'stim_high']
   
    return design

In [22]:
tr = 2
n_subjects = 1
n_runs = 3

save_dir = '../designs'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
    
for subject_id in np.hstack([np.arange(1,n_subjects+1), 'DEBUG']):
    designs_this_session = []
    print(subject_id)
    is_practice = subject_id == 'PRACTICE'
    is_debug = subject_id == 'DEBUG'
    
    for run in range(1,n_runs*2+1):
        settings = get_settings(tr=tr)
        include_cue, stim_set = get_task_type(run, subject_id, is_practice=is_practice, is_debug=is_debug)
        print('Subject: %s, run: %d, cue: %s' %(subject_id, run, include_cue))
        design = generate_block_design(settings['n_trials'], settings['jitter'], ps=settings['ps'],
                                       stim_sets=stim_set,
                                       mr_design=False, include_cue=include_cue)
        design['block'] = run
        designs_this_session.append(design)
    
    design = pd.concat(designs_this_session)
    design = get_stimuli(subject_id=subject_id, design=design)
    fn = 'sub-' + str(subject_id).zfill(2) + '_design'
    print(fn)
    design.to_csv(save_dir + '/' + fn + '.csv', sep='\t', index_label='trial_ID')

1
Settings:

        Sessions: 2
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 1, run: 1, cue: False
Settings:

        Sessions: 2
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 1, run: 2, cue: False
Settings:

        Sessions: 2
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 1, run: 3, cue: False
Settings:

        Session

In [23]:
design

,stimulus_set,correct_stim_lr,cue,p_win_left,p_win_right,p_win_correct,p_win_incorrect,phase_1,phase_2,phase_3,phase_4,phase_5,phase_6,phase_7,block,stim_left,stim_right,stim_high,stim_low
0,0,1,SPD,0.2,0.8,0.8,0.2,-0.0001,0.7500,-0.0001,2,0.5,0.75,0.735535,1,B,A,A,B
1,2,0,SPD,0.65,0.35,0.65,0.35,-0.0001,0.7500,-0.0001,2,0.5,0.75,0.726781,1,M,N,M,N
2,0,0,ACC,0.8,0.2,0.8,0.2,-0.0001,0.7500,-0.0001,2,0.5,0.75,0.709440,1,A,B,A,B
3,2,0,ACC,0.65,0.35,0.65,0.35,-0.0001,0.7500,-0.0001,2,0.5,0.75,0.879458,1,M,N,M,N
4,0,1,ACC,0.2,0.8,0.8,0.2,-0.0001,0.7500,-0.0001,2,0.5,0.75,0.693607,1,B,A,A,B
5,1,0,SPD,0.7,0.3,0.7,0.3,-0.0001,0.7500,-0.0001,2,0.5,0.75,0.744880,1,J,K,J,K
6,0,1,ACC,0.2,0.8,0.8,0.2,-0.0001,0.7500,-0.0001,2,0.5,0.75,0.824124,1,B,A,A,B
7,1,0,SPD,0.7,0.3,0.7,0.3,-0.0001,0.7500,-0.0001,2,0.5,0.75,0.842174,1,J,K,J,K
8,0,1,SPD,0.2,0.8,0.8,0.2,-0.0001,0.7500,-0.0001,2,0.5,0.75,0.746597,1,B,A,A,B
9,1,1,SPD,0.3,0.7,0.7,0.3,-0.0001,0.7500,-0.0001,2,0.5,0.75,0.878063,1,K,J,J,K


## For debug, we create a nice a short session

In [24]:
tr = 2
n_subjects = 1
n_runs = 3

save_dir = '../designs'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
    
for subject_id in np.hstack(['DEBUG']):
    designs_this_session = []
    is_practice = subject_id == 'PRACTICE'
    is_debug = subject_id == 'DEBUG'
    
    for run in range(1,n_runs*2+1):
        settings = get_settings(tr=tr)
        include_cue, stim_set = get_task_type(run, subject_id, is_practice=is_practice, is_debug=is_debug)
        print('Subject: %s, run: %d, cue: %s' %(subject_id, run, include_cue))
        design = generate_block_design(6, settings['jitter'], ps=settings['ps'],
                                       stim_sets=stim_set,
                                       mr_design=False, include_cue=include_cue)
        design['block'] = run
        designs_this_session.append(design)
    
    design = pd.concat(designs_this_session)
    design = get_stimuli(subject_id=subject_id, design=design)
    fn = 'sub-' + str(subject_id).zfill(2) + '_design'
    print(fn)
    design.to_csv(save_dir + '/' + fn + '.csv', sep='\t', index_label='trial_ID')

Settings:

        Sessions: 2
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: DEBUG, run: 1, cue: True
Settings:

        Sessions: 2
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: DEBUG, run: 2, cue: True
Settings:

        Sessions: 2
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: DEBUG, run: 3, cue: True
Settings:

        

In [25]:
design

,stimulus_set,correct_stim_lr,cue,p_win_left,p_win_right,p_win_correct,p_win_incorrect,phase_1,phase_2,phase_3,phase_4,phase_5,phase_6,phase_7,block,stim_left,stim_right,stim_high,stim_low
0,2,0,SPD,0.65,0.35,0.65,0.35,-0.0001,0.7500,-0.0001,2,0.5,0.75,0.748380,1,M,N,M,N
1,1,0,ACC,0.7,0.3,0.7,0.3,-0.0001,0.7500,-0.0001,2,0.5,0.75,0.744114,1,J,K,J,K
2,0,1,SPD,0.2,0.8,0.8,0.2,-0.0001,0.7500,-0.0001,2,0.5,0.75,0.635012,1,B,A,A,B
3,2,1,SPD,0.35,0.65,0.65,0.35,-0.0001,0.7500,-0.0001,2,0.5,0.75,0.805868,1,N,M,M,N
4,1,0,SPD,0.7,0.3,0.7,0.3,-0.0001,0.7500,-0.0001,2,0.5,0.75,0.519092,1,J,K,J,K
5,2,0,ACC,0.65,0.35,0.65,0.35,-0.0001,0.7500,-0.0001,2,0.5,0.75,0.733054,1,M,N,M,N
0,2,1,ACC,0.35,0.65,0.65,0.35,-0.0001,0.7500,-0.0001,2,0.5,0.75,0.587164,2,N,M,M,N
1,2,1,SPD,0.35,0.65,0.65,0.35,-0.0001,0.7500,-0.0001,2,0.5,0.75,0.730717,2,N,M,M,N
2,1,0,ACC,0.7,0.3,0.7,0.3,-0.0001,0.7500,-0.0001,2,0.5,0.75,0.733385,2,J,K,J,K
3,0,0,ACC,0.8,0.2,0.8,0.2,-0.0001,0.7500,-0.0001,2,0.5,0.75,0.646945,2,A,B,A,B


### Idem for the practice session

In [26]:
tr = 2
n_runs = 3

save_dir = '../designs'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
    
for subject_id in np.hstack(['PRACTICE']):
    designs_this_session = []
    is_practice = subject_id == 'PRACTICE'
    is_debug = subject_id == 'DEBUG'
    
    for run in range(1,n_runs*2+1):
        settings = get_settings(tr=tr)
        include_cue, stim_set = get_task_type(run, subject_id, is_practice=is_practice, is_debug=is_debug)
        print('Subject: %s, run: %d, cue: %s' %(subject_id, run, include_cue))
        design = generate_block_design(6, settings['jitter'], ps=settings['ps'],
                                       stim_sets=stim_set,
                                       mr_design=False, include_cue=include_cue, practice=True)
        design['block'] = run
        designs_this_session.append(design)
    
    design = pd.concat(designs_this_session)
    design = get_stimuli(subject_id=subject_id, design=design)
    fn = 'sub-' + str(subject_id).zfill(2) + '_design'
    print(fn)
    design.to_csv(save_dir + '/' + fn + '.csv', sep='\t', index_label='trial_ID')

Settings:

        Sessions: 2
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: PRACTICE, run: 1, cue: False
Settings:

        Sessions: 2
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: PRACTICE, run: 2, cue: False
Settings:

        Sessions: 2
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: PRACTICE, run: 3, cue: False
Setting

In [28]:
design

,stimulus_set,correct_stim_lr,cue,p_win_left,p_win_right,p_win_correct,p_win_incorrect,phase_1,phase_2,phase_3,phase_4,phase_5,phase_6,phase_7,block,stim_left,stim_right,stim_high,stim_low
0,2,0,,0.65,0.35,0.65,0.35,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.793451,1,u,n,u,n
1,1,1,,0.3,0.7,0.7,0.3,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.696271,1,y,s,s,y
2,0,1,,0.2,0.8,0.8,0.2,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.864061,1,m,j,j,m
3,0,0,,0.8,0.2,0.8,0.2,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.739490,1,j,m,j,m
4,1,0,,0.7,0.3,0.7,0.3,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.860721,1,s,y,s,y
5,2,1,,0.35,0.65,0.65,0.35,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.530810,1,n,u,u,n
0,2,1,,0.35,0.65,0.65,0.35,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.694672,2,n,u,u,n
1,1,1,,0.3,0.7,0.7,0.3,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.873939,2,y,s,s,y
2,0,0,,0.8,0.2,0.8,0.2,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.517833,2,j,m,j,m
3,1,0,,0.7,0.3,0.7,0.3,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.580318,2,s,y,s,y


In [230]:
idx = design.block == 2

In [29]:
idx 

0    False
1    False
2    False
3    False
0     True
1     True
2     True
3     True
0    False
1    False
2    False
3    False
0    False
1    False
2    False
3    False
0    False
1    False
2    False
3    False
0    False
1    False
2    False
3    False
Name: block, dtype: bool

0    False
1    False
2    False
3    False
0     True
1    False
2    False
3    False
0    False
1    False
2    False
3    False
0    False
1    False
2    False
3    False
0    False
1    False
2    False
3    False
0    False
1    False
2    False
3    False
Name: block, dtype: bool

In [32]:
1 < np.cumsum(idx) < 2

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [22]:
idx = design.block == 1
#idx & np.cumsum(idx) < 4

In [273]:
idx = design.block==1
design.loc[idx].groupby(['stimulus_set'])[['stim_left', 'stim_right']].last().reset_index()[['stim_left', 'stim_right']].values.tolist()

[['m', 'j'], ['s', 'y'], ['n', 'u']]

In [248]:
design[['stim_left', 'stim_right']].values

array([['u', 'n'],
       ['y', 's'],
       ['n', 'u'],
       ['s', 'y'],
       ['j', 'm'],
       ['m', 'j'],
       ['s', 'y'],
       ['j', 'm'],
       ['y', 's'],
       ['m', 'j'],
       ['u', 'n'],
       ['n', 'u'],
       ['n', 'u'],
       ['m', 'j'],
       ['s', 'y'],
       ['u', 'n'],
       ['y', 's'],
       ['j', 'm'],
       ['n', 'u'],
       ['u', 'n'],
       ['s', 'y'],
       ['m', 'j'],
       ['j', 'm'],
       ['n', 'u'],
       ['y', 's'],
       ['s', 'y'],
       ['y', 's'],
       ['n', 'u'],
       ['m', 'j'],
       ['s', 'y'],
       ['m', 'j'],
       ['s', 'y'],
       ['y', 's'],
       ['m', 'j'],
       ['j', 'm'],
       ['s', 'y']], dtype=object)

In [236]:
design['stim_right'].uni

array(['n', 's', 'u', 'y', 'm', 'j'], dtype=object)